In [12]:
import keras
import pandas as pd
import numpy as np 
import re

In [3]:
df = pd.read_csv("../data/Sentiment.csv/Sentiment.csv")

In [4]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [25]:
df.shape

(13871, 21)

In [5]:
data = df[["sentiment", "text"]]

In [6]:
data.head()

,sentiment,text
0,Neutral,RT @NancyLeeGrahn: How did everyone feel about...
1,Positive,RT @ScottWalker: Didn't catch the full #GOPdeb...
2,Neutral,RT @TJMShow: No mention of Tamir Rice and the ...
3,Positive,RT @RobGeorge: That Carly Fiorina is trending ...
4,Positive,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...


In [7]:
data.sentiment.unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [8]:
data.sentiment.value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [10]:
data.text = data.text.apply(lambda x: x.lower())

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [13]:
data.text = data.text.apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
data.text = data.text.apply(lambda x: x.replace("rt", ""))

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [20]:
data.text[:10]

0     nancyleegrahn how did everyone feel about the...
1     scottwalker didnt catch the full gopdebate la...
2     tjmshow no mention of tamir rice and the gopd...
3     robgeorge that carly fiorina is trending  hou...
4     danscavino gopdebate w realdonaldtrump delive...
5     gregabbott_tx tedcruz on my first day i will ...
6     warriorwoman91 i liked her and was happy when...
7    going on msnbc live with thomasarobes around 2...
8    deer in the headlights  lizzwinstead ben carso...
9     nancyosborne180 last nights debate proved it ...
Name: text, dtype: object

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import to_categorical

In [22]:
max_feature = 2000
tokenizer = Tokenizer(num_words=max_feature, split=" ")

In [23]:
tokenizer.fit_on_texts(data.text)

In [24]:
X = tokenizer.texts_to_sequences(data.text)

In [27]:
X = pad_sequences(X)

In [30]:
X.shape

(13871, 28)

In [31]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   52,   78,  338,  449,   22,    2,  413,
        361,   95,   29,   51, 1023,    1])

In [32]:
word_index = tokenizer.word_index

In [33]:
len(word_index)

19460

In [36]:
for index, i in enumerate(word_index.items()):
    print(i)
    if index> 10:
        break

('gopdebate', 1)
('the', 2)
('gopdebates', 3)
('to', 4)
('of', 5)
('is', 6)
('a', 7)
('and', 8)
('i', 9)
('in', 10)
('rwsurfergirl', 11)
('you', 12)


In [37]:
from keras.losses import categorical_crossentropy

In [38]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_feature, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [45]:
Y = pd.get_dummies(data.sentiment)

In [ ]:
model.fit(X, Y, epochs=10, batch_size= 64)

Epoch 1/10
13871/13871 [==============================] - 53s 4ms/step - loss: 0.6340 - acc: 0.7344
Epoch 2/10
13871/13871 [==============================] - 47s 3ms/step - loss: 0.6073 - acc: 0.7452
Epoch 3/10
13871/13871 [==============================] - 48s 3ms/step - loss: 0.5883 - acc: 0.7522
Epoch 4/10
13871/13871 [==============================] - 51s 4ms/step - loss: 0.5619 - acc: 0.7615
Epoch 5/10
 5952/13871 [===========>..................] - ETA: 32s - loss: 0.5252 - acc: 0.7812